In [1]:
import pandas as pd 

In [ ]:
# Generate different inputs and/or adapt the original one 
df = pd.read_excel('input/RG_PVP_07.02.2023_BNL_AMCO.xlsx')

#df = df[0:99]
df = df.loc[1389:]

df[['procedura', 'anno']] = df['ruolo'].str.split('/', expand=True)

# Add 'Tribunale di ' to the beginning of each value in the 'TRIBUNALE' column
df['TRIBUNALE'] = 'Tribunale di ' + df['TRIBUNALE'].astype(str)

df = df[['TRIBUNALE', 'procedura', 'anno']] 

print(df)

#CHANGE THE NAMEEEEEEEE
df.to_excel('input/1389toEND.xlsx', index=False)

In [20]:
#If different inputs were used, see the combined amount makes sense
done1 = pd.read_excel('output_toEND.xlsx')

done2 = pd.read_excel('output/outputTO1388.xlsx')
len(done1['id'].unique()) + len(done2['id'].unique())

846

In [46]:
# JOIN THE GIVEN INPUT WITH THE OUTPUT (IN CASE EXTRA COLUMNS WERE GIVEN)
from source.functions import dicToExcel, common_mistakes
df = pd.read_excel('input/RG_PVP_07.02.2023_BNL_AMCO.xlsx')
df[['procedure', 'anno']] = df['ruolo'].str.split('/', expand=True)

# Add 'Tribunale di ' to the beginning of each value in the 'TRIBUNALE' column
df['TRIBUNALE'] = 'Tribunale di ' + df['TRIBUNALE'].astype(str)

df['TRIBUNALE'] = df['TRIBUNALE'].astype(str).str.lower()
df['TRIBUNALE'] = df['TRIBUNALE'].map(common_mistakes)

output = pd.read_excel('output/output_toEND.xlsx')
output[['TRIBUNALE', 'procedure', 'anno']] = output['id'].str.split('_', expand=True)

joined_output = df.merge(output, on=['TRIBUNALE', 'procedure', 'anno'], how='outer')

cols = output.columns.drop(['TRIBUNALE', 'procedure', 'anno'])
df_with_output = joined_output.dropna(subset=cols, how='all')
df_without_output = joined_output[joined_output[cols].isna().all(axis=1)]

#Comment the next three lines to do the check that's below
order = ['num_prat', 'codiceproc', 'procedura', 'original ruolo', 'ruolo','TRIBUNALE', 'PORTAFOGLIO', 'Lotto', 'Data Vendita', 'Offerta minima', 'Rialzo minimo', "Prezzo base d'asta", 'Tipologia', 'Address']
df_with_output = df_with_output[order]
df_without_output = df_without_output[df.columns.drop(['procedure', 'anno'])]

with pd.ExcelWriter('output/output_file.xlsx', engine='xlsxwriter') as writer:
    # Write df_with_output to a sheet named 'procedure_valide'
    df_with_output.to_excel(writer, sheet_name='procedure_valide', index=False)
    
    # Access the xlsxwriter workbook and worksheet objects for 'procedure_valide' sheet
    worksheet_with_output = writer.sheets['procedure_valide']
    
    # Adjust the column widths in the 'procedure_valide' sheet
    for i, col in enumerate(df_with_output.columns):
        max_length = max(df_with_output[col].astype(str).apply(len).max(), len(col))
        worksheet_with_output.set_column(i, i, max_length)
    
    # Write df_without_output to a new sheet named 'procedure_vuote'
    df_without_output.to_excel(writer, sheet_name='procedure_vuote', index=False)
    
    # Access the xlsxwriter workbook and worksheet objects for 'procedure_vuote' sheet
    worksheet_without_output = writer.sheets['procedure_vuote']
    
    # Adjust the column widths in the 'procedure_vuote' sheet
    for i, col in enumerate(df_without_output.columns):
        max_length = max(df_without_output[col].astype(str).apply(len).max(), len(col))
        worksheet_without_output.set_column(i, i, max_length)
    

In [45]:
# Check if any outputs were left out
df_first= pd.read_excel('output/output_toEND.xlsx')
df_sec= pd.read_excel('output/output_file.xlsx')

print(df_first[~df_first['id'].isin(df_sec['id'])]['id'])

Series([], Name: id, dtype: object)


In [12]:
# IN CASE OF ERROR IN THE DOWNLOAD SCRAPING:
from date_utils import today
import os 
from source.functions import read_or_create_tribunales, common_mistakes
TRIBUNALE_PATH = os.environ.get("TRIBUNALE_PATH")
today = '08_02_2024'
df = pd.read_excel('input/startTO1388.xlsx')

df = df[['TRIBUNALE', 'procedura', 'anno']] 
df['TRIBUNALE'] = df['TRIBUNALE'].astype(str).str.lower()
df['TRIBUNALE'] = df['TRIBUNALE'].map(common_mistakes)


directory_path = f'output/{today}/made'
loaded_dict = read_or_create_tribunales(TRIBUNALE_PATH)
made = pd.DataFrame()

for file in os.listdir(directory_path):
    tribunale_len, procedura, anno = map(str, file.rstrip('.html').split('_'))
    tribunale_len = len(tribunale_len)
    n_tribunale, procedura, anno = map(int, file.rstrip('.html').split('_'))
    n_tribunale = f'{n_tribunale:0{tribunale_len}d}'  # I put back the 0s it might have taken from the start in the conversion
    tribunale = list(loaded_dict.keys())[list(loaded_dict.values()).index(n_tribunale)]
    add = pd.DataFrame([(tribunale, procedura, anno)], columns=['TRIBUNALE', 'procedura', 'anno'])
    made = pd.concat([made, add], ignore_index=True)

for file in os.listdir(f'output/{today}/empty'):
    tribunale_len, procedura, anno = map(str, file.rstrip('.html').split('_'))
    tribunale_len = len(tribunale_len)
    n_tribunale, procedura, anno = map(int, file.rstrip('.html').split('_'))
    n_tribunale = f'{n_tribunale:0{tribunale_len}d}'  # I put back the 0s it might have taken from the start in the conversion
    tribunale = list(loaded_dict.keys())[list(loaded_dict.values()).index(n_tribunale)]
    add = pd.DataFrame([(tribunale, procedura, anno)], columns=['TRIBUNALE', 'procedura', 'anno'])
    made = pd.concat([made, add], ignore_index=True)

#prev = pd.read_excel('prev.xlsx')
made['TRIBUNALE'] = made['TRIBUNALE'].astype(str).str.lower()
df = df.sort_values(by=['procedura', 'anno'])  # replace 'your_sort_column' with the actual column name
made = made.sort_values(by=['procedura', 'anno'])  # replace 'your_sort_column' with the actual column name

# Resetting the index after sorting
df = df.reset_index(drop=True)
made = made.reset_index(drop=True)

df['procedura'] = df['procedura'].astype(str)
df['anno'] = df['anno'].astype(str)
made['procedura'] = made['procedura'].astype(str)
made['anno'] = made['anno'].astype(str)

# Merge the DataFrames
merged_df = pd.merge(df, made, how='outer', indicator=True).loc[lambda x : x['_merge'] != 'both']


print("Differences:")
print(merged_df)
merged_df.to_excel('differences.xlsx')

Differences:
                              TRIBUNALE procedura  anno      _merge
0                     tribunale di roma   0000051  2024   left_only
1                 tribunale di velletri   0000468  2023   left_only
5            tribunale di vibo valentia         1  2024   left_only
8     tribunale di macerata ex camerino        10  2012   left_only
17                   tribunale di prato       101  2022   left_only
...                                 ...       ...   ...         ...
1407           tribunale di napoli nord       399  2023  right_only
1408              tribunale di velletri       468  2023  right_only
1409                 tribunale di forli      1489  2021  right_only
1410               tribunale di perugia      2878  2015  right_only
1411               tribunale di perugia      5261  2014  right_only

[145 rows x 4 columns]
